In [1]:
from langchain.llms import AzureOpenAI
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

### **Initialize the LLM model which is deployed in Azure with LangChain**

In [2]:

def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=400,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm


### **Add personality to the model and ask questions**
We call directly the Azure OpenAI API with ***ChatCompletion*** API

In [3]:
#prepare prompt
messages=[{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer in a clear and concise manner."},
          { "role": "user", "content": "Good morning, how are you today?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)
display (HTML("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content))


In [4]:

"""
If you try to run following code, you will get an error:
openai.error.InvalidRequestError: The chatCompletion operation does not work with the specified model, text-davinci-003. 
Please choose a different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.
"""
#Error!
#answer = openai.ChatCompletion.create(engine = "text-davinci-003",
#                                   messages = messages,)

'\nIf you try to run following code, you will get an error:\nopenai.error.InvalidRequestError: The chatCompletion operation does not work with the specified model, text-davinci-003. \nPlease choose a different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.\n'

In [4]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are q HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
          { "role": "user", "content": "What's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)

#print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)

display(HTML(answer.choices[0].message.content))


In [5]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer as for a FIVE YEARS old child."},
          { "role": "user", "content": "what's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)

#print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)
display(HTML(answer.choices[0].message.content))



### **LangChain**

LangChain is a framework built around Large Language Models (LLMs).

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs.

In [6]:
#model "gpt-35-turbo"  
#You can see that gpt-35-turbo has been trained in QnA conversational style.
llm=init_llm()
answer=llm("Good morning, how are you?")
display (HTML("gpt-35-turbo: " + answer))

In [7]:
#model "text-davinci-003"
#text-davinci-003 is a more generic model, trained for the mode: text-in, text-out
llm=init_llm(OPENAI_DAVINCI_MODEL_NAME, OPENAI_DAVINCI_DEPLOYMENT_NAME)
answer=llm("Good morning, how are you?")
display(HTML("text-davinci-003: "+ answer))

In [8]:
llm=init_llm()
answer=llm("Create a Python function that takes a string argument and reverses it.")
display (HTML("gpt-35-turbo: " + answer))

In [9]:
llm=init_llm(OPENAI_DAVINCI_MODEL_NAME, OPENAI_DAVINCI_DEPLOYMENT_NAME)
answer=llm("Create a Python function that takes a string argument and reverses it.")
display (HTML("text-davinci-003: " + answer))

### **Prompt Engineering**

In [10]:
from langchain import PromptTemplate

#create template for prompt

template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

llm=init_llm()
prompt = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="How do you assess the risk tolerance of a new client?"))
display (HTML("gpt-35-turbo: " + answer))


In [11]:
#asking not related question
prompt = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="What's the fastest car in the world?"))
display (HTML("gpt-35-turbo: " + answer))

### **Using LLMChain** 

LLMChain is a simplest LangChain chain.

In [12]:
from langchain import LLMChain

#default llm is gpt-35-turbo
llm=init_llm()

chain = LLMChain(llm=llm, prompt=prompt)

ans = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you assess the risk tolerance of a new client, and how do you use this information in recommending trading strategies?")
display (HTML(ans))

###  **One-shot, Few-shot learning**

This technique could improve model performance by a lot. 
We can use the model to learn from a few examples and then use it to generate text. This is called few-shot learning. We can also use the model to learn from a single example and then use it to generate text. This is called one-shot learning.

In [13]:
template_few_shot = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that a user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
           
            USER: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.
            
            USER: Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I'd recommended that a client diversify their portfolio further to reduce risk exposure.
            
            USER: How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
            ASSISTANT: I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance. 
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """
           

In [14]:
prompt_few_shot = PromptTemplate(template=template_few_shot, input_variables=["profession", "expertise", "question"])
chain = LLMChain(llm=llm, prompt=prompt_few_shot)

res=chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you use technology or specific financial tools to assist in risk management for your clients?")
display (HTML(res))


##### Since we don't save a history of the conversation, the model will fail to answer questions that require context.

In [15]:
res = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "Which software do you use?")
display (HTML(res))

#### **LangChain Few-Shot learning**

In [16]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate


# create few shot examples
examples = [
    {
        "query": "How do you assess the risk tolerance of a new client?",
        "answer": "I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.."
    }, 
    
    {
        "query": "Can you provide an example of a specific risk management strategy you've recommended to a client in a volatile market situation?",
        "answer": "During the market volatility caused by the pandemic, I recommended that a client diversify their portfolio further to reduce risk exposure."
    },
    {
        "query": "How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?",
        "answer": "I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance."
    },
    
]

# create a example template
example_template = """
User: {query}
Assistant: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix_template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: It's not clear or the question is not related to {expertise}.
examples: 
"""
# and the suffix our user input and output indicator
suffix_template = """
User: {query}
Assistant: """

prefix_prompt = PromptTemplate(input_variables=["profession", "expertise"], template=prefix_template)

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix = prefix_prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management"),
    suffix=suffix_template,
    input_variables=["query"],
    example_separator="\n\n"
)


In [17]:
query="How do you use technology or specific financial tools to assist in risk management for your clients?"
display (HTML(few_shot_prompt_template.format(query=query)))

In [18]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
ans = chain.run(few_shot_prompt_template.format(query=query))
display (HTML(ans))

### **Retain conversation history** 

##### Previously we saw that a model fails to answer the question that requires context. We can solve this problem by retaining the conversation history. We can do this by using the LangChain ConversationBufferMemory.

In [19]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:

            <|im_end|>
            """

In [20]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.schema import HumanMessage

llm=init_llm()

prompt_few_shot = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [21]:
ans = conversation.run (input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="How do you use technology or specific financial tools to assist in risk management for your clients?"))

display (HTML(ans))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:

> Finished chain.


In [22]:
ans = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="Which software do you use?"))

display (HTML(ans))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:  Hello! I'm happy to answer your question. There are many ways that technology and financial tools

In [23]:
print(f"conversation history: {conversation.memory}")
history = conversation.memory.chat_memory.messages
for msg in history:
    print ( f"{msg.type}: {msg.content}")
    

conversation history: chat_memory=ChatMessageHistory(messages=[HumanMessage(content="You are a Financial Trading Consultant answering users questions. \n            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.\n            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.\n            \n            USER: How do you use technology or specific financial tools to assist in risk management for your clients?\n            ASSISTANT:\n            \n            <|im_end|>\n            ", additional_kwargs={}, example=False), AIMessage(content=" Hello! I'm happy to answer your question. There are many ways that technology and financial tools can be used to assist in risk management. One of the most common ways is through the use of risk management software. This software can help to identify potential risks, assess the 

In [24]:
print(conversation.prompt)

input_variables=['history', 'input'] output_parser=None partial_variables={} template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:' template_format='f-string' validate_template=True


In [25]:
input=[]
history = conversation.generate(input)
print(history)


#for conversation in history:
#    print(f"{conversation['sender']}: {conversation['message']}")

IndexError: list index out of range

In [23]:
ans = conversation.run (input = prompt.format(profession="Financial Trading Consultant",
                        expertise="Risk Management", 
                            question="List all questions I've asked you about Risk Management?"))
display (HTML(ans))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:  Hello! I'm happy to answer your question. There are many ways that technology and financial tools

#### TODO there other conversation memory types. Put them in a table and provide examples